In [32]:
import requests as re
import pandas as pd
import json
import os
import pprint as pp
# Load API Keys from OS environmental variables
fs_client_id = os.environ["FOURSQUARE_CLIENT_ID"]
fs_client_secret = os.environ["FOURSQUARE_CLIENT_SECRET"]
y_API_key = os.environ["YELP_API_KEY"]

# Define number of venues to index
num_venues = 10

# Define API URLs with corresponding parametrs 
fs_url = 'https://api.foursquare.com/v2/venues/explore'
fs_params = {'client_id' : fs_client_id, 'client_secret' : fs_client_secret, 'll' : '43.6491,-79.3796',
            'radius' : 1500, 'section' : 'food', 'v' : '20210114', 'limit' : num_venues}
y_url = 'https://api.yelp.com/v3/businesses/search'
y_params = {'term' : 'food', 'latitude' : 43.6491, 'longitude' : -79.3796, 'radius' : 1500 , 
            'limit' : num_venues, 'sort_by' : 'rating', 'attributes' : 'hot_and_new'}
y_headers = {'Authorization' : f'Bearer {y_API_key}'}

# Make API calls
fs_response = re.get(fs_url, params=fs_params).json()
#y_response = re.get(y_url, params=y_params, headers=y_headers)

#pp.pprint(fs_response)
# with open('response.json') as f:
#     response_list = json.load(f)
# print(type(response_list))
#fs_response_list = json.load(fs_response)

# Convert to Pandas Dataframes
#pd.json_normalize(fs_response)

In [42]:
print(type(fs_response))
pd.json_normalize(fs_response)

<class 'dict'>


,meta.code,meta.requestId,response.suggestedFilters.header,response.suggestedFilters.filters,response.headerLocation,response.headerFullLocation,response.headerLocationGranularity,response.query,response.totalResults,response.suggestedBounds.ne.lat,response.suggestedBounds.ne.lng,response.suggestedBounds.sw.lat,response.suggestedBounds.sw.lng,response.groups
0,200,6000c53d5d7adf267b6dbeac,Tap to show:,"[{'name': 'Open now', 'key': 'openNow'}]",Financial District,"Financial District, Toronto",neighborhood,food,129,43.6626,-79.360978,43.6356,-79.398222,"[{'type': 'Recommended Places', 'name': 'recom..."


In [33]:
for i in range(0,num_venues):
    pp.pprint(fs_response['response']['groups'][0]['items'][i]['venue']['location']['formattedAddress'])

['140 Yonge St (at Temperance St)', 'Toronto ON M5C 1X6', 'Canada']
['Commerce Court East', 'Toronto ON', 'Canada']
['1 Richmond Street West (Yonge Street)', 'Toronto ON', 'Canada']
['66 Wellington St West (at Bay Street)', 'Toronto ON M5K 1H6', 'Canada']
['100 King St W (at Bay St.)', 'Toronto ON', 'Canada']
['1959 Queen St E', 'Toronto ON M4L 1H7', 'Canada']
['56 Temperance Street', 'Toronto ON M5H 3V5', 'Canada']
['18 King St. E (at Yonge St.)', 'Toronto ON M5C 1C4', 'Canada']
['1 King St W (Yonge St)', 'Toronto ON', 'Canada']
['10 Temperance St fl 5 (Yonge St)', 'Toronto ON M5H 1Y4', 'Canada']


In [35]:
for i in range(0,num_venues):
    pp.pprint(fs_response['response']['groups'][0]['items'][i]['venue']['name'])

'Dineen Coffee'
'Mos Mos Coffee'
'Richmond Station'
'Canoe'
'Brick Street Bakery'
'Hogtown Smoke'
'John & Sons Oyster House'
'Beerbistro'
'Café Plenty'
'The Chase'
